In [1]:
import numpy as np

from strat import MACDandEMA

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=3000,
)

In [9]:
macd_strat = MACDandEMA(
    long_short="long",
    ema_length=np.arange(100, 301, 100),
    fast_length=np.arange(10, 21, 10),
    macd_below=np.array([0]),
    signal_smoothing=np.array([10]),
    slow_length=np.arange(20, 31, 10),
)

In [10]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.1, 0.25]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [11]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 9
Total order settings to test: 16
Total combinations of settings to test: 144
Total candles: 3,000
Total candles to test: 432,000


In [12]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
37,2,5,8.0,5,3,27.630,62.500,0.859,35.734,1276.301,276.301
45,2,13,8.0,5,3,24.468,62.500,0.846,30.459,1244.685,244.685
46,2,14,7.0,5,2,32.505,71.429,0.834,43.161,1325.045,325.045
141,8,13,10.0,6,4,42.206,60.000,0.826,81.448,1422.056,422.056
36,2,4,8.0,5,3,21.577,62.500,0.825,35.290,1215.765,215.765
47,2,15,7.0,5,2,22.503,71.429,0.818,42.926,1225.033,225.033
44,2,12,8.0,5,3,19.394,62.500,0.811,30.254,1193.937,193.937
142,8,14,9.0,6,3,56.195,66.667,0.805,95.262,1561.949,561.949
143,8,15,9.0,6,3,44.358,66.667,0.794,94.436,1443.583,443.583
138,8,10,10.0,6,4,43.061,60.000,0.773,93.123,1430.613,430.613
